In [8]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx
import numpy as np

# 1. Carregar o vídeo original
video = VideoFileClip("Trabalhocg-VEED.mp4")

# 2. Processar vídeo: inverter imagem a cada 20 segundos
clips = []
for i in range(0, int(video.duration), 20):
    clip = video.subclip(i, min(i + 20, video.duration))
    if (i // 20) % 2 == 1:
        clip = clip.fx(vfx.mirror_x)  # Inverter horizontalmente
    clips.append(clip)

processed_video = concatenate_videoclips(clips)

# 3. Cortar trechos e reorganizar
main_part = processed_video.subclip(0, 60)
cut_part = processed_video.subclip(60, 80)
remaining_part = processed_video.subclip(80, processed_video.duration)
final_video = concatenate_videoclips([main_part, remaining_part, cut_part])

# 4. Reduzir o som gradativamente
audio = final_video.audio
duration = final_video.duration

def fade_audio(get_frame, t):
    frame = get_frame(t)  # Obtém o áudio no momento t
    if isinstance(t, (int, float)):
        if t >= (duration - 10):
            fade_factor = 0
        else:
            fade_factor = max(0, 1 - (t // 30) * 0.1)
    else:  # Caso t seja um array NumPy (para múltiplos frames)
        fade_factor = 1 - (t // 30) * 0.2
        fade_factor = np.clip(fade_factor, 0, 1)  # Garante valores entre 0 e 1
    
    # Ajusta o fade_factor para ter a mesma forma que o frame
    if len(frame.shape) > 1:  # Áudio com mais de um canal (ex.: estéreo)
        fade_factor = fade_factor[:, np.newaxis]
    
    return frame * fade_factor

def mute_last_10_seconds(get_frame, t):
    frame = get_frame(t)  # Obtém o áudio no momento t
    if isinstance(t, (int, float)) and t >= (duration - 10):
        return frame * 0
    elif isinstance(t, np.ndarray):
        mute_factor = np.where(t >= (duration - 10), 0, 1)
        if len(frame.shape) > 1:  # Áudio com mais de um canal (ex.: estéreo)
            mute_factor = mute_factor[:, np.newaxis]
        return frame * mute_factor
    return frame

faded_audio = audio.fl(fade_audio).fl(mute_last_10_seconds)
final_video = final_video.set_audio(faded_audio)

# 5. Renderizar e salvar o vídeo final
final_video.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
